In [1]:
import qiime2
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, metadata, feature_table, alignment, phylogeny, diversity, emperor, feature_classifier, taxa, composition

# Create directories
!mkdir -p data

# ENTER YOUR WORKING DIRECTORY HERE
# Like this (on linux): workdir = '/dir/'

workdir = '/mnt/e/dev/pda'
datadir = '/mnt/d/data/ena_files'

%cd $workdir

/mnt/e/dev/pda


In [ ]:
import os, sys
import pandas as pd

df = pd.read_csv("run_prefix.txt", sep="\t")
prefix = df[["sample_name","run_prefix"]]

# listing directories
file_list = os.listdir(os.getcwd())

for file in file_list:
    if file[:4] == "1629":
        sample_id = file[:18]
        run_prefix = prefix[prefix['sample_name'] == sample_id]['run_prefix'].tolist()[0]
        src = sample_id + ".fastq.gz"
        dst = sample_id + run_prefix[3:15]+"001.fastq.gz"
        os.rename(src, dst)
        print(src)
        print(dst)